# **Projet IA & Cognition: A decision Support System for Project Management based on Ontology Learning**

## 1.1-  **Partie I: pre-processing and parsing data from unstructured text**





#### **Extraction de données à partir de PMBOK (en particulier project cost,schedule and scope management)**

In [ ]:
pip install PyPDF2

In [ ]:
import PyPDF2
fhandle = open(r'PMBOKGuide_5th_Ed-128-252.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(fhandle)
pagehandle = pdfReader.getPage(0)
print(pagehandle.extractText())

102©2013 Project Management Institute.
 A Guide to the Project Management Body of Knowledge (PMBOK
® Guide) Œ Fifth Edition 
4 - PROJECT INTEGRATION MANAGEMENT
4.6.1 Close Project or Phase: Inputs4.6.1.1 Project Management PlanDescribed in Section 4.2.3.1. The project management plan becomes the agreement between the project 
manager and project sponsor, de˜ning what constitutes project completion.
4.6.1.2 Accepted DeliverablesDescribed in Section 5.5. Accepted deliverables may include approved product speci˜cations, delivery 
receipts, and work performance documents. Partial or interim deliverables may also be included for phased or 

cancelled projects.4.6.1.3 Organizational Process AssetsDescribed in Section 2.1.4. The organizational process assets that can in˚uence the Close Project or Phase 
process include, but are not limited to:
 Project or phase closure guidelines or requirements (e.g., administrative procedures, project audits, 

project evaluations, and transition criteria);

In [ ]:
with open('PMBOKGuide_5th_Ed-131-252.pdf','rb') as pdf_file, open('131_252.txt', 'w') as text_file:
    read_pdf = PyPDF2.PdfFileReader(pdf_file)
    number_of_pages = read_pdf.getNumPages()
    for page_number in range(number_of_pages):   # use xrange in Py2
        page = read_pdf.getPage(page_number)
        page_content = page.extractText()
        text_file.write(page_content)

#### **Segmentation des données : les noms, les verbes, les synonymes, etc**

In [ ]:
import requests 
import string
import os
import matplotlib
import pandas as pd
from google.colab import files
import re
from nltk.corpus import wordnet
import numpy as np  
import nltk
import io
from gensim import corpora
import gensim     
from gensim.utils import simple_preprocess   
from gensim import corpora, models
from gensim.parsing.preprocessing import STOPWORDS
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
import pickle
import spacy
from tqdm import tqdm

In [ ]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9MB 1.2MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp36-none-any.whl size=829180944 sha256=45bfad6625c2c6428095421e11952273241b41d9d84a1397bb508082ea477166
  Stored in directory: /tmp/pip-ephem-wheel-cache-xnb0wm0i/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [ ]:
# !python -m spacy download en_core_web_lg
nlp = spacy.load("en_core_web_lg")

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### **Read the TXT file of the corpus** 

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
myfile = open("/content/drive/My Drive/Project_OWL/Corpus.txt")
st=myfile.read()

In [ ]:
st

'102©2013 Project Management Institute.\n A Guide to the Project Management Body of Knowledge (PMBOK\n® Guide) Œ Fifth Edition \n4 - PROJECT INTEGRATION MANAGEMENT\n4.6.1 Close Project or Phase: Inputs4.6.1.1 Project Management PlanDescribed in Section 4.2.3.1. The project management plan becomes the agreement between the project \nmanager and project sponsor, de˜ning what constitutes project completion.\n4.6.1.2 Accepted DeliverablesDescribed in Section 5.5. Accepted deliverables may include approved product speci˜cations, delivery \nreceipts, and work performance documents. Partial or interim deliverables may also be included for phased or \n\ncancelled projects.4.6.1.3 Organizational Process AssetsDescribed in Section 2.1.4. The organizational process assets that can in˚uence the Close Project or Phase \nprocess include, but are not limited to:\n Project or phase closure guidelines or requirements (e.g., administrative procedures, project audits, \n\nproject evaluations, and transit

###**Breakdown of texts into sentences and Removal of punctuation and unnecessary words**

In [ ]:
doc = nlp(st)
sentences=[]
for sent in doc.sents:
  sentences.append(str (sent)) 
sentences[:10]

['102©2013 Project Management Institute.\n ',
 'A Guide to the Project Management Body of Knowledge (PMBOK\n',
 '® Guide) Œ Fifth Edition \n4 - PROJECT INTEGRATION MANAGEMENT\n',
 '4.6.1 Close Project or Phase:',
 'Inputs4.6.1.1 Project Management PlanDescribed in Section 4.2.3.1.',
 'The project management plan becomes the agreement between the project \nmanager and project sponsor, de˜ning what constitutes project completion.\n',
 '4.6.1.2',
 'Accepted DeliverablesDescribed in Section 5.5.',
 'Accepted deliverables may include approved product speci˜cations, delivery \nreceipts, and work performance documents.',
 'Partial or interim deliverables may also be included for phased or \n\ncancelled projects.4.6.1.3 Organizational Process AssetsDescribed in Section 2.1.4.']

In [ ]:
##Remove words that contains no letters and stopwords too
exclude = set(string.punctuation)
def Clean(s):
    s=re.sub(r'\b[0-9]+\b\s*', '', str (s))
    s=re.sub(r'[^\w]', ' ', s)
    return ''.join(ch for ch in s if ch not in exclude)

In [ ]:
result = list(map(lambda x: Clean(x), sentences))
print(len(result))

3977


In [ ]:
result[400:450]

['The stakeholder register is used to identify stakeholders who can provide  information on the requirements ',
 'The stakeholder register also captures major requirements and main expectations   stakeholders may have for the project  ',
 'Licensed',
 'To ',
 'Jorge Diego Fuentes Sanchez PMI MemberID  2399412This copy is a PMI Member benefit  not for distribution  sale  or reproduction  Project Management Institute   ',
 'A Guide to the Project Management Body of Knowledge  PMBOK ',
 '  Guide  Œ',
 'Fifth Edition  ',
 '  PROJECT SCOPE MANAGEMENT5  Collect Requirements ',
 'Tools and Techniques5   Interviews ',
 'An interview is a formal or informal approach to elicit information from stakeholders by talking to them directly   ',
 'It is typically performed by asking prepared and spontaneous questions and recording the responses ',
 'Interviews   are often conducted on an individual basis between an interviewer and an interviewee  but may involve multiple   interviewers and or multiple 

In [ ]:
 sentences[400:401]

['The stakeholder register is used to identify stakeholders who can provide \ninformation on the requirements.']

In [ ]:
## We Will use spacy for the next part for both entities recognition and relations selection
def WordsTags(sen):
  sentsDetails=[]
  for token in nlp (sen):
    s="Word : {:}, POS: {:}".format(token, token.pos_)
    sentsDetails.append(s)
  return sentsDetails

list(map(lambda x: WordsTags(x), sentences[400:403]))

[['Word : The, POS: DET',
  'Word : stakeholder, POS: NOUN',
  'Word : register, POS: NOUN',
  'Word : is, POS: AUX',
  'Word : used, POS: VERB',
  'Word : to, POS: PART',
  'Word : identify, POS: VERB',
  'Word : stakeholders, POS: NOUN',
  'Word : who, POS: PRON',
  'Word : can, POS: VERB',
  'Word : provide, POS: VERB',
  'Word : \n, POS: SPACE',
  'Word : information, POS: NOUN',
  'Word : on, POS: ADP',
  'Word : the, POS: DET',
  'Word : requirements, POS: NOUN',
  'Word : ., POS: PUNCT'],
 ['Word : The, POS: DET',
  'Word : stakeholder, POS: NOUN',
  'Word : register, POS: NOUN',
  'Word : also, POS: ADV',
  'Word : captures, POS: VERB',
  'Word : major, POS: ADJ',
  'Word : requirements, POS: NOUN',
  'Word : and, POS: CCONJ',
  'Word : main, POS: ADJ',
  'Word : expectations, POS: NOUN',
  'Word : \n\n, POS: SPACE',
  'Word : stakeholders, POS: NOUN',
  'Word : may, POS: VERB',
  'Word : have, POS: AUX',
  'Word : for, POS: ADP',
  'Word : the, POS: DET',
  'Word : project, PO

## 1.2- **Partie II: Modeling a domain Ontology: OWL Ontology and finding inference rules.**

###**Entity Recognition and Taxonomic relations selection**

In [ ]:
##EntityRecognition using Chunking Method of spacy, we also add an algorithm to concatinate the consecutive nouns not well appreciated by spacy##
def EntityFinder(doc):
  Entities=[]
  indPass=False
  Entities1=[]
  for chunk in doc.noun_chunks:
      Entities.append(chunk.text)
  for (ind) in range(len(Entities)):
    if indPass==True:
      indPass=False
      continue
    id_sequence = map(lambda x: x.orth, [token for token in doc])
    textList = map(lambda x: nlp.vocab[x].text, [id for id in id_sequence])
    Newst = " ".join(w for w in textList)
    if (ind==len(Entities)-1):
      res=''
    else:
      res = Newst.split(str (Entities[ind]), maxsplit=1)[-1]\
                .split(maxsplit=1)[0]
    if (res in ['of', 'and','for']) and (ind<len(Entities)-1):
      Entity=str (Entities[ind])+' '+res+' '+str (Entities[ind+1])
      Entities1.append(Entity)
      Newst=Newst.replace(str (Entities[ind]), '',1)
      indPass=True
      doc =nlp(Newst)
    else:
      Entities1.append(str (Entities[ind]))
  for ind1 in range(len(Entities1)):
    if (len (str (Entities1[ind1])) > 1):
      if((str (Entities1[ind1])[0]=='a') &(str (Entities1[ind1])[1] ==' ')):
        Entities1[ind1]=Entities1[ind1].replace(Entities1[ind1][0],'',1)
        Entities1[ind1]=Entities1[ind1].replace(Entities1[ind1][0],'',1)
  return Entities1


In [ ]:
strr='Acceptance of the code is essential for project managers, and is a requirement for the following PMI® exams. OPM is a strategy execution framework utilizing project'
Working_Examples=nlp('Acceptance of the code is essential for project managers, and is a requirement for the following PMI® exams. OPM is a strategy execution framework utilizing project')
SentencesWithRules=[]
for sent in Working_Examples.sents:
  SentencesWithRules.append(sent)

list(map(lambda x: EntityFinder(x) , SentencesWithRules))

[['Acceptance of the code',
  'project managers',
  'requirement for the following PMI® exams'],
 ['OPM', 'strategy execution framework', 'project']]

In [ ]:
word=' is a '
indexStart = strr.find(word)
indexEnd= indexStart+ len(word)-1
indexEnd
wordIndex=[indexStart,indexEnd]

print(indexStart,indexEnd)

61 66


In [ ]:
### TBOX: --Relations Taxonomiques--  search for relations between classes and lower classes: using ' is a ' for equivalence relation and ' part of ' for inclusion relation

def search_Classes_SubClasses(sentence1,choice):
  try:
    print(sentence1)
    Relations=[]
    sentence =str (sentence1)
    EntitiesFounded=EntityFinder(sentence1)
    indexStart = sentence.find(choice)
    while (indexStart != -1):
      gg=-1
      for ele in EntitiesFounded:
        if((str (ele)[0]=='a') &(str (ele) ==' ')):
          str(ele)[0]=''

        indexStart1 = sentence.find(str (ele))
        diffMin=indexStart-indexStart1
        if(diffMin < 0):
          break
        gg= gg+ 1
      object1=EntitiesFounded[gg]
      object2=EntitiesFounded[gg+1]

      if (choice==' is a '):
        relation=[{"Relation Taxonomique": 'Equivalence', "Object1": object1, "Object2": object2,'RelationFound':'is a'}] #object1 type object2
        Relations.append(relation)
      elif (choice==' part of '):
        relation=[{"Relation Taxonomique": 'Inclusion', "Object1": object1, "Object2": object2,'RelationFound':'part of'}] #object1 subclass object2
        Relations.append(relation)
      
      if ((choice ==' including ') | (choice==' such as ')):
        boucleInd=True
        newSen=sentence[indexStart:]
        while (boucleInd):
          Entities=EntityFinder(nlp(newSen))
          ent1Ind= newSen.find(Entities[0])
          indexe=ent1Ind+len(Entities[0])

          if (indexe < len(newSen)):
            if (newSen[indexe]!=','):
              boucleInd=False
          else:
            boucleInd=False
          if ((choice ==' including ')):
            relation=[{"Relation Taxonomique": 'Equivalence', "Object1": object1, "Object2": Entities[0],'RelationFound': 'including'}] #object2 subclass object1
          if ((choice ==' such as ')):
            relation=[{"Relation Taxonomique": 'Equivalence', "Object1": object1, "Object2": Entities[0],'RelationFound': 'such as'}] #object2 type object1
          Relations.append(relation)
          newSen=newSen.replace(Entities[0], ' ',1)

      
      sentence=sentence.replace(choice, ' ',1)

      indexStart = sentence.find(choice)

    return Relations
  except:
    print('exception')

In [ ]:
sen=nlp('includes consideration of attributes such as resource experience and/or skill level, as well as various ')
search_Classes_SubClasses(sen,' such as ')

includes consideration of attributes such as resource experience and/or skill level, as well as various 


[[{'Object1': 'consideration of attributes',
   'Object2': 'resource experience',
   'Relation Taxonomique': 'Equivalence',
   'RelationFound': 'such as'}]]

In [ ]:
sen=nlp('TCPIThe value of the work planned to be  completed to a point in time  usually ')
search_Classes_SubClasses(sen,' is a ')

TCPIThe value of the work planned to be  completed to a point in time  usually 


[]

In [ ]:
####Test TBOX Function on some sentences of the corpus to see the taxonomic relations 'is a'
IsaRelations = list(map(lambda x: search_Classes_SubClasses(nlp(x),' is a ') , sentences)) 
for i in IsaRelations:
  print(i)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Project management plan  
 updates

.5
Project documents  

 
updates

.6
Organizational process  

 
assets updates

Figure 7-10. 

Control Costs:
Inputs, 
Tools & Techniques, and 
OutputsProject Cost Management7.4ControlCostsP

sP

sC
W

Cost



sProjectDocumentsProject
t

Wo


4.3Direct
andManage ProjectWork

4.2Develop
ProjectManagementPlanEnterprise/

Organization4.4Monitor
andControl
ProjectWork

4.5Perform

IntegratedChange Control7.3Determine

BudgetFigure 7-11. 

Control Costs Data Flow
DiagramLicensed
To:
Jorge Diego Fuentes Sanchez PMI MemberID: 2399412This copy is a PMI Member benefit, not for distribution, sale, or reproduction.216©2013 Project Management Institute.
 
A Guide to the Project Management Body of Knowledge (PMBOK

® Guide) Œ Fifth Edition 

7 - PROJECT COST MANAGEMENTUpdating
the budget requires knowledge of the actual costs spent to date.
Any increase to the authorized 
budget can o

méme chose pour les relations d'inclusion

In [ ]:
PartOfRelations = list(map(lambda x: search_Classes_SubClasses(nlp(x),' part of ') , sentences))
for i in PartOfRelations:
  if (i !=[]):
    print(i)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
The start-to-˜nish relationship 
is very rarely used but is included to present a complete list of the PDM relationship types.

Activity
A
Activity
BActivity A

Activity A
Activity BActivity BActivity A
Activity BFinish to Start (FS)

Start to Finish (SF)

Start to Start (SS)

Finish to Finish (FF)

Figure 6-9.
Precedence 
Diagramming Method (PDM) Relationship Types6.3.2.2 Dependency 

DeterminationDependencies may be characterized by the following attributes: mandatory or discretionary, internal or external, 
as described below.
Dependency has four attributes, but two can be applicable at the same time in following 

ways: mandatory external dependencies, mandatory internal dependencies, discretionary external dependencies, 

or discretionary internal dependencies.
 
Mandatory dependencies. 

Mandatory dependencies are those that are legally or contractually required 
or inherent in the nature of the work.
M

In [ ]:
SuchAsRelations = list(map(lambda x: search_Classes_SubClasses(nlp(x),' such as ') , sentences))
for i in SuchAsRelations:
  if (i !=[]):
    print(i)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
during the process of sequencing the activities.
Mandatory dependencies should not be confused with 

assigning schedule constraints in the scheduling tool.
Licensed To: Jorge Diego Fuentes
Sanchez PMI MemberID: 2399412This copy is a PMI Member benefit, not for distribution, sale, or reproduction.158©2013 Project Management Institute.
 
A Guide to the Project Management Body of Knowledge (PMBOK

® Guide) Œ
Fifth Edition 

6 - PROJECT TIME MANAGEMENT Discretionary dependencies. 

Discretionary dependencies are sometimes referred to as preferred logic, 
preferential logic, or soft logic.
Discretionary dependencies are established based on knowledge of best 

practices within a particular application area or some unusual aspect of the project where a speci˜c 

sequence is desired, even though there may be other acceptable sequences.
Discretionary dependencies 

should be fully documented since they can create ar

In [ ]:
IncludingRelations = list(map(lambda x: search_Classes_SubClasses(nlp(x),' including ') , sentences))
for i in IncludingRelations:
  if (i !=[]):
    print(i)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.

Start to Finish (SF)

Start to Start (SS)

Finish to Finish (FF)

Figure 6-9.
Precedence 
Diagramming Method (PDM) Relationship Types6.3.2.2 Dependency 

DeterminationDependencies may be characterized by the following attributes: mandatory or discretionary, internal or external, 
as described below.
Dependency has four attributes, but two can be applicable at the same time in following 

ways: mandatory external dependencies, mandatory internal dependencies, discretionary external dependencies, 

or discretionary internal dependencies.
 
Mandatory dependencies. 

Mandatory dependencies are those that are legally or contractually required 
or inherent in the nature of the work.
Mandatory dependencies often involve physical limitations, such 

as on a construction project, where it is impossible to erect the superstructure until after the foundation 

has been built, or on an electronics project, where a proto

In [ ]:
IncludingRelations
SuchAsRelations
PartOfRelations
IsaRelations

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [[{'Object1': '2399412This copy',
    'Object2': 'PMI Member benefit',
    'Relation Taxonomique': 'Equivalence',
    'RelationFound': 'is a'}]],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [[{'Object1': '2399412This copy',
    'Object2': 'PMI Member benefit',
    'Relation Taxonomique': 'Equivalence',
    'RelationFound': 'is a'}]],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [[{'Object1': '2399412This copy',
    'Object2': 'PMI Member benefit',
    'Relation Taxonomique': 'Equivalence',
    'RelationFound': 'is a'}]],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [[{'Object1': '2399412This copy',
    'Object2': 'PMI Member benefit',
    'Relation Taxonomique': 'Equivalence',
    'RelationFound': 'is a'}]],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [

In [ ]:
PartOfRelations1=[]
for ele in PartOfRelations:
  if (ele!=[]):
    for i in ele:
      for ji in i:
        PartOfRelations1.append(ji)
del PartOfRelations1[2]

In [ ]:
PartOfRelations1

[{'Object1': 'assumptions',
  'Object2': 'part of their planning process',
  'Relation Taxonomique': 'Inclusion',
  'RelationFound': 'part of'},
 {'Object1': 'the supporting contract WBS',
  'Object2': 'part of the contracted work',
  'Relation Taxonomique': 'Inclusion',
  'RelationFound': 'part of'},
 {'Object1': 'the part of the budget',
  'Object2': 'the ﬁknown-unknownsﬂ',
  'Relation Taxonomique': 'Inclusion',
  'RelationFound': 'part of'},
 {'Object1': 'funding requirements',
  'Object2': 'part of the overall project budget',
  'Relation Taxonomique': 'Inclusion',
  'RelationFound': 'part of'}]

In [ ]:
IsaRelations1=[]
i=0
for ele in IsaRelations:
  if ((ele!=[]) & (ele is not None)):
    for i in ele:
      for ji in i:
        if ((str (ji['Object1'])!='2399412This copy') & (str (ji['Object1'])!='It')):
          IsaRelations1.append(ji)

In [ ]:
SuchAsRelations1=[]
i=0
for ele in SuchAsRelations:
  if ((ele!=[]) & (ele is not None)):
    for i in ele:
      for ji in i:
        if ((str (ji['Object1'])!='others') & (str (ji['Object1'])!='whole') & (str (ji['Object1'])!='them') & (str (ji['Object1'])!='different types')):
          SuchAsRelations1.append(ji)

In [ ]:
IncludingRelations1=[]
i=0
for ele in IncludingRelations:
  if ((ele!=[]) & (ele is not None)):
    for i in ele:
      for ji in i:
        if ((str (ji['Object1'])!='many sources')):
          IncludingRelations1.append(ji)

In [ ]:
### We founded some important taxonomic relations for our context like :
## TCPI ->measure of the cost performance, Storyboarding -> prototyping technique,
## The scope management plan -> the project management plan, WBS Data Flow Diagram -> hierarchical decomposition of the total scope
## From the avove rules we will take the the most relevent ones 

### **Non Taxonomic relations selection, ABOX**

In [ ]:
### ABOX : Extract non taxonomic relations 

### First, We will extract relations from the verbs and build a relation from the before and after the verb
ListWords = ["NOUN", "PROPN","PUNCT","CCONJ","NUM","DET", "SCONJ"]
def extract_Relation(text:spacy.tokens.doc.Doc, token:spacy.tokens.token.Token):
  Verb = token
  avant = []
  apres= []
  verbeAAvant = text[token.i -1]  
  while ((verbeAAvant.pos_ not in ListWords) and (verbeAAvant.pos_ in ["AUX", "ADP", "ADV","PART","VERB"])) : 
    avant.append(verbeAAvant)
    token = verbeAAvant
    verbeAAvant = text[token.i -1]
  token = Verb
  if (token.i +1 < len(text)):
    verbeApres = text[token.i +1]
    while ((verbeApres.pos_ not in ListWords) and (verbeApres.pos_ in ["AUX", "ADP", "ADV","PART","VERB"]) and (token.i +1 < len(text))) : 
      apres.append(verbeApres)
      token = verbeApres
      if (token.i +1 >= len(text)):
        return ({"NNTaxoRelation":  avant+[Verb]+apres})
      verbeApres = text[token.i +1]
  else:
    return ({"NNTaxoRelation":  avant+[Verb]+apres}) 
  avant.reverse()
  return ({"NNTaxoRelation":  avant+[Verb]+apres})  

In [ ]:
def Extractions_per_verbs (text:spacy.tokens.doc.Doc):
  results = []
  for token in text:

    if token.pos_ == "VERB": 
      if any(token in sublist for sublist in list(map(lambda x: x["NNTaxoRelation"],results))):
        continue 
      results.append(extract_Relation(text, token))
  return results

In [ ]:
### Search for groups of relations based on verbs 
#Example
sen=nlp('A technique that utilizes a decision matrix to provide a systematic   analytical approach for establishing criteria  such as risk levels  uncertainty  and valuation  to evaluate   and rank many ideas  ')

dd=list(map(lambda x: x["NNTaxoRelation"],Extractions_per_verbs(sen)))
dd


[[utilizes], [to, provide], [for, establishing], [to, evaluate], [rank]]

In [ ]:
def convertTostr(res):
  res1=[]
  for i in res:
    hh=[str (ji) for ji in i]
    res1.append(hh)
  return res1
def resList(s):
  for p in range(len(s)):
    news=''
    for k in s[p]:
      news=news+ ' '+ k
    s[p]=news
  return s

In [ ]:
s=convertTostr(dd)
print(resList(s))

[' utilizes', ' to provide', ' for establishing', ' to evaluate', ' rank']


In [ ]:
EntityFinder(sen)

['A technique',
 'decision matrix',
 'systematic   analytical approach',
 'criteria',
 'risk levels',
 'uncertainty and valuation',
 'many ideas']

In [ ]:
###We used those two functions above to serach for full triplets composed by NN group as concept1, relation group and then NN group that follow The Rel group 

In [ ]:
def Triple(text:spacy.tokens.doc.Doc):
  triples=[]
  for sen in text:
    Entities=EntityFinder(sen) 
    if len(Entities) >1:
      for ind in range(len(Entities)):
        object1=Entities[ind]
        if (ind+1 < len(Entities)):
          object2=Entities[ind+1]

          indexStartObject1 = str (sen).find(str (object1))
          indexStartObject2 = str (sen).find(str (object2))
          LenObj2 = len(str (object2))
          ss=str (sen)
          NewSt=ss[indexStartObject1:(indexStartObject2+LenObj2)]
          NewSt1=nlp(NewSt)
          reslts=list(map(lambda x: x["NNTaxoRelation"],Extractions_per_verbs(NewSt1)))
          resltsStr=convertTostr(reslts)
          print(resltsStr)
          nnn=resList(resltsStr)
          triple={'Object 1':object1,'Object1_Lemmatization':list (w.lemma_ for w in nlp(str (object1) )),'Relation':nnn,'Object2':object2,'Object2_Lemmatization':list (w.lemma_ for w in nlp(str (object2) )) }
          triples.append(triple)
        else:
          break
    else:
      continue
  return triples

In [ ]:
#Test of function in an example from the corpus
txt=nlp('A technique used to generate and collect multiple ideas related to project and product  requirements')
lk=[]
for sent in txt.sents:
  lk.append(sent)
res =Triple(lk) 
res

[['used', 'to', 'generate'], ['collect']]
[['related', 'to']]


[{'Object 1': 'A technique',
  'Object1_Lemmatization': ['a', 'technique'],
  'Object2': 'multiple ideas',
  'Object2_Lemmatization': ['multiple', 'idea'],
  'Relation': [' used to generate', ' collect']},
 {'Object 1': 'multiple ideas',
  'Object1_Lemmatization': ['multiple', 'idea'],
  'Object2': 'project and product',
  'Object2_Lemmatization': ['project', 'and', 'product'],
  'Relation': [' related to']}]

In [ ]:
for inno in res:
  if inno['Relation'] ==[]:
    print(inno['Object 1'])

Recherche de tous les triplets

In [ ]:
### Run The Triple Function over all sentences to collect all triples of concepts and relations.
AllTriples=[]
for sen in sentences:
  txt =nlp(str (sen) )
  lk=[]
  for sent in txt.sents:
    lk.append(sent)
  res=Triple(lk)
  print(res)
  AllTriples.append(res)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
[{'Object 1': 'Bottom-Up EstimatingBottom-up estimating', 'Object1_Lemmatization': ['Bottom', '-', 'up', 'EstimatingBottom', '-', 'up', 'estimating'], 'Relation': [' of estimating'], 'Object2': 'method of estimating project duration', 'Object2_Lemmatization': ['method', 'of', 'estimate', 'project', 'duration']}, {'Object 1': 'method of estimating project duration', 'Object1_Lemmatization': ['method', 'of', 'estimate', 'project', 'duration'], 'Relation': [' of estimating'], 'Object2': 'cost', 'Object2_Lemmatization': ['cost']}, {'Object 1': 'cost', 'Object1_Lemmatization': ['cost'], 'Relation': [' by aggregating'], 'Object2': 'the estimates of the \nlower-level components', 'Object2_Lemmatization': ['the', 'estimate', 'of', 'the', '\n', 'low', '-', 'level', 'component']}, {'Object 1': 'the estimates of the \nlower-level components', 'Object1_Lemmatization': ['the', 'estimate', 'of', 'the', '\n', 'low', '-', 'l

In [ ]:
AllTriples

[[],
 [{'Object 1': 'A Guide',
   'Object1_Lemmatization': ['a', 'Guide'],
   'Object2': 'the Project Management Body of Knowledge',
   'Object2_Lemmatization': ['the',
    'Project',
    'Management',
    'Body',
    'of',
    'knowledge'],
   'Relation': []},
  {'Object 1': 'the Project Management Body of Knowledge',
   'Object1_Lemmatization': ['the',
    'Project',
    'Management',
    'Body',
    'of',
    'knowledge'],
   'Object2': 'PMBOK',
   'Object2_Lemmatization': ['PMBOK'],
   'Relation': []}],
 [{'Object 1': '® Guide',
   'Object1_Lemmatization': ['®', 'Guide'],
   'Object2': 'Fifth Edition \n4 - PROJECT INTEGRATION MANAGEMENT',
   'Object2_Lemmatization': ['Fifth',
    'Edition',
    '\n',
    '4',
    '-',
    'project',
    'integration',
    'management'],
   'Relation': []}],
 [{'Object 1': '4.6.1 Close Project',
   'Object1_Lemmatization': ['4.6.1', 'Close', 'Project'],
   'Object2': 'Phase',
   'Object2_Lemmatization': ['phase'],
   'Relation': []}],
 [{'Object 1':

In [ ]:
## Select the triples with relations not null
FinalTriples=[]

for ele in AllTriples:
  resTriples= list(filter(lambda x: (x['Relation']!=[]) & (x['Object 1']!='it') & (x['Object 1']!='they') & (x['Object 1']!='them') & (x['Object 1']!='˜nd ways')
   & (x['Object2']!='it') & (x['Object2']!='they') & (x['Object2']!='them') & (x['Object2']!='˜nd ways'),ele))
  FinalTriples.append(resTriples)
FinalTriples[10:15]

[[{'Object 1': 'The organizational process assets',
   'Object1_Lemmatization': ['the', 'organizational', 'process', 'asset'],
   'Object2': 'the Close Project',
   'Object2_Lemmatization': ['the', 'Close', 'Project'],
   'Relation': [' can in˚uence']},
  {'Object 1': 'process',
   'Object1_Lemmatization': ['process'],
   'Object2': 'Project',
   'Object2_Lemmatization': ['project'],
   'Relation': [' can in˚uence']},
  {'Object 1': 'Project',
   'Object1_Lemmatization': ['project'],
   'Object2': 'phase closure guidelines',
   'Object2_Lemmatization': ['phase', 'closure', 'guideline'],
   'Relation': [' include', ' are not limited to']},
  {'Object 1': 'Historical information and lessons',
   'Object1_Lemmatization': ['historical', 'information', 'and', 'lesson'],
   'Object2': 'knowledge base',
   'Object2_Lemmatization': ['knowledge', 'base'],
   'Relation': [' learned']},
  {'Object 1': 'information',
   'Object1_Lemmatization': ['information'],
   'Object2': 'risk management',
   

In [ ]:
##Allconcepts contains both object1 and object2
AllConcepts=[]
for ind in FinalTriples:
  for j in ind:
    AllConcepts.append(j['Object2'])
len(AllConcepts)

2149

In [ ]:
word_counter = {}
for word in AllConcepts:
     if word in word_counter:
       word_counter[word] += 1
     else:
       word_counter[word] = 1
popular_words = sorted(word_counter, key = word_counter.get, reverse = True) 
top_700 = popular_words[:700]


In [ ]:
top_700[:30]

['the project',
 'Figure',
 'the data',
 'the schedule model',
 'information',
 'the schedule',
 'Section',
 'the work',
 'work',
 'time',
 'procedures',
 'detail',
 'stakeholders',
 'the WBS',
 'the project team',
 'activities',
 'each activity',
 'what',
 'change',
 'historical information',
 'the activity',
 'conjunction',
 'the cost baseline',
 'requirements',
 'changes',
 'the project schedule model',
 'project',
 'an activity',
 'variances',
 'given point']

### **OWL file Creation**

In [ ]:
from rdflib.namespace import DC, DCTERMS, DOAP, FOAF, OWL, RDF, RDFS, SKOS, VOID, XMLNS, XSD
from rdflib import URIRef, BNode, Literal, Namespace, Graph
from rdflib.extras import describer

In [ ]:
g= Graph()
g.bind("owl",OWL)
g.bind("pr","http://pmbox.com/h#")#Bind prefix to namespace
ns_url = "http://pmbox.com/h#"
# Create an RDF URI node to use as the subject for multiple triples
g.add((URIRef('http://pmbox.com/h'), RDF.type, OWL.Ontology ))

#### **ajout de Relations taxonomiques**

In [ ]:
RltsT=IncludingRelations1+SuchAsRelations1+IsaRelations1+PartOfRelations1
RltsT[0:5]

[{'Object1': 'The term project scope',
  'Object2': 'product scope',
  'Relation Taxonomique': 'Equivalence',
  'RelationFound': 'including'},
 {'Object1': 'the project scope',
  'Object2': 'product scope',
  'Relation Taxonomique': 'Equivalence',
  'RelationFound': 'including'},
 {'Object1': 'Consultants;Stakeholders',
  'Object2': 'customers',
  'Relation Taxonomique': 'Equivalence',
  'RelationFound': 'including'},
 {'Object1': 'project',
  'Object2': 'project and product scope',
  'Relation Taxonomique': 'Equivalence',
  'RelationFound': 'including'},
 {'Object1': 'all product and project work',
  'Object2': 'the project management work',
  'Relation Taxonomique': 'Equivalence',
  'RelationFound': 'including'}]

In [ ]:
for c in RltsT:
    cl = URIRef(ns_url+c['Object1'].replace(" ","_"))
    cl1 =URIRef(ns_url+c['Object2'].replace(" ","_"))
    if (c['RelationFound']=='is a'):
      g.add((cl, RDF.type, cl1))    # add tag
    if (c['RelationFound']=='including'):
      g.add((cl1, RDFS.subClassOf, cl))
    if (c['RelationFound']=='part of'):
      g.add((cl, RDFS.subClassOf, cl1))
    if (c['RelationFound']=='such as'):
      g.add((cl1, RDF.type, cl))

#### **Ajout des relations non Taxonomiques**

In [ ]:
for triple in FinalTriples:
  if triple!=[]:
    for ele in triple:
      dom = URIRef(ns_url+ele['Object 1'].replace(" ","_"))
      range = URIRef(ns_url+ele['Object2'].replace(" ","_"))
      if (ele['Object2'] not in top_700):
        #Data Property
        rel = URIRef(ns_url+ele['Relation'][0].replace(" ","_"))
        g.add((rel,RDF.type,OWL.DatatypeProperty))
        g.add((rel,RDFS.domain,dom))
        g.add((rel,RDFS.range,Literal(str (ele['Object2']))))
      else:
        #Object Property
        rel = URIRef(ns_url+ele['Relation'][0].replace(" ","_"))
        g.add((rel,RDF.type,OWL.ObjectProperty))
        g.add((rel,RDFS.domain,dom))
        g.add((rel,RDFS.range,range))       




In [ ]:
import owlready2 as owl
ont = owl.get_ontology("http://project.org/pmbok.owl") 

In [ ]:
with open('/content/drive/My Drive/Project_OWL/ontology.txt', 'w', encoding='utf-8') as output:
    output.write(g.serialize(format="pretty-xml").decode("utf-8"))

In [ ]:
with open('/content/drive/My Drive/Project_OWL/ontology.txt', 'r') as testwritefile:
    print(testwritefile.read())

<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
  xmlns:ns1="http://pmbox.com/h#signi˜"
  xmlns:pr="http://pmbox.com/h#"
  xmlns:owl="http://www.w3.org/2002/07/owl#"
  xmlns:ns2="http://pmbox.com/h#the_working_

"
  xmlns:ns3="http://pmbox.com/h#˜nance_

"
  xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#"
  xmlns:ns4="http://pmbox.com/h#manual,"
  xmlns:ns5="http://pmbox.com/h#the_Project_Scope_

"
  xmlns:ns6="http://pmbox.com/h#limited_amount_of_detailed_

"
  xmlns:ns7="http://pmbox.com/h#˜"
  xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
  xmlns:ns8="http://pmbox.com/h#speci˜"
  xmlns:ns9="http://pmbox.com/h#limited_amount_of_detailed_
"
  xmlns:ns10="http://pmbox.com/h#actual_start_and_˜"
  xmlns:ns11="http://pmbox.com/h#various_analytical_
"
  xmlns:ns12="http://pmbox.com/h#an_estimate_for_activity_

"
  xmlns:ns13="http://pmbox.com/h#numerous_general_˜"
  xmlns:ns14="http://pmbox.com/h#previous,_

"
>
  <owl:DatatypeProperty rdf:about="http://pmbox.com/h#_can_provid

### **Rules of inference**

In [ ]:
#nous faisons d'abord une recherche par ligne, lorsque nous trouverons les relations si alors nous nous concentrerons sur la partie intermédiaire entre si et alors et la dernière partie également.
#Pour le cas de as a result of nous prenons en considération toute la phrase divisée en deux avant et après le type, puis nous chercherons les relations taxonomiques existantes dans les phrases trouvées et formerons une liste finale qui comprend toutes les règles.
def searchRules(text:spacy.tokens.doc.Doc):
  rules=[]
  for sen in text.sents:
    st=str (sen)
    lk=[]
    newsent1=''
    ruleType=''
    newsent2=''
    stlow=st.lower()
    if (('if' in stlow) & ('then' in stlow)):
      pos = stlow.index('if')
      pos1=stlow.index('then')
      ruleType='IfThen'
      newsent1=stlow[pos+2:pos1]
      newsent2=stlow[pos1+4:]
    if ('as a result of' in stlow.lower()):
      len1=len('as a result of')
      pos = stlow.index('as a result of')
      ruleType='AsaResultOf'
      newsent1=stlow[:pos]
      newsent2=stlow[pos+len1:]
    lk.append(nlp(newsent1))
    res1=Triple(lk)
    lk.clear()
    lk.append(nlp(newsent2))
    res2=Triple(lk)
    if(ruleType!=''):
      if ((res1==[]) & (res2==[])):
        triple={'RuleType':ruleType,'Cause':newsent1,'Result':newsent2} if ruleType=='IfThen' else {'RuleType':ruleType,'Cause':newsent2,'Result':newsent1}
      elif ((res1!=[])&(res2!=[])):
        triple={'RuleType':ruleType,'Cause':res1,'Result':res2} if ruleType=='IfThen' else {'RuleType':ruleType,'Cause':res2,'Result':res1}
      elif ((res1==[])&(res2!=[])):
        triple={'RuleType':ruleType,'Cause':newsent1,'Result':res2} if ruleType=='IfThen' else {'RuleType':ruleType,'Cause':res2,'Result':newsent1}
      else:
        triple={'RuleType':ruleType,'Cause':res1,'Result':newsent2} if ruleType=='IfThen' else {'RuleType':ruleType,'Cause':newsent2,'Result':res1}
      rules.append(triple)   
  return rules

In [ ]:
#Two rules were generated for the two types result of and if..then 
searchRules(nlp('Project documents that may be updated as a result of the Perform Integrated Change. If the approved change requests have an effect on the project scope, then the scope statement, the WBS, and the WBS dictionary are revised'))

[['integrated']]
[['approved']]
[]
[]
[]


[{'Cause': [{'Object 1': ' the perform',
    'Object1_Lemmatization': [' ', 'the', 'perform'],
    'Object2': 'integrated change',
    'Object2_Lemmatization': ['integrate', 'change'],
    'Relation': [' integrated']}],
  'Result': 'project documents that may be updated ',
  'RuleType': 'AsaResultOf'},
 {'Cause': [{'Object 1': ' the approved change requests',
    'Object1_Lemmatization': [' ', 'the', 'approve', 'change', 'request'],
    'Object2': 'an effect',
    'Object2_Lemmatization': ['an', 'effect'],
    'Relation': [' approved']},
   {'Object 1': 'an effect',
    'Object1_Lemmatization': ['an', 'effect'],
    'Object2': 'the project scope',
    'Object2_Lemmatization': ['the', 'project', 'scope'],
    'Relation': []}],
  'Result': [{'Object 1': ' the scope statement',
    'Object1_Lemmatization': [' ', 'the', 'scope', 'statement'],
    'Object2': 'the wbs',
    'Object2_Lemmatization': ['the', 'wbs'],
    'Relation': []},
   {'Object 1': 'the wbs',
    'Object1_Lemmatization': [

In [ ]:
### Run The Triple Function over all sentences to collect all triples of concepts and relations.
AllRules=[]
for sen in sentences:
  res=searchRules(nlp(sen))
  print(res)
  AllRules.append(res)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[['include'], ['are', 'not', 'limited', 'to'], ['licensed', 'to']]
[{'RuleType': 'AsaResultOf', 'Cause': [{'Object 1': ' the close project', 'Object1_Lemmatization': [' ', 'the', 'close', 'project'], 'Relation': [], 'Object2': 'phase process', 'Object2_Lemmatization': ['phase', 'process']}, {'Object 1': 'phase process', 'Object1_Lemmatization': ['phase', 'process'], 'Relation': [' include', ' are not limited to', ' licensed to'], 'Object2': 'jorge diego fuentes sanchez pmi memberid', 'Object2_Lemmatization': ['jorge', 'diego', 'fuentes', 'sanchez', 'pmi', 'memberid']}], 'Result': [{'Object 1': 'organizational process assets', 'Object1_Lemmatization': ['organizational', 'process', 'asset'], 'Relation': [], 'Object2': 'updatesthe organizational process assets', 'Object2_Lemmatization': ['updatesthe', 'organizational', 'process', 'asset']}]}]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[

In [ ]:
ls=filter(lambda x: x!=[],AllRules)
FinalRules=list(ls)
len(FinalRules)

10

In [ ]:
FinalRules

[[{'Cause': [{'Object 1': ' the close project',
     'Object1_Lemmatization': [' ', 'the', 'close', 'project'],
     'Object2': 'phase process',
     'Object2_Lemmatization': ['phase', 'process'],
     'Relation': []},
    {'Object 1': 'phase process',
     'Object1_Lemmatization': ['phase', 'process'],
     'Object2': 'jorge diego fuentes sanchez pmi memberid',
     'Object2_Lemmatization': ['jorge',
      'diego',
      'fuentes',
      'sanchez',
      'pmi',
      'memberid'],
     'Relation': [' include', ' are not limited to', ' licensed to']}],
   'Result': [{'Object 1': 'organizational process assets',
     'Object1_Lemmatization': ['organizational', 'process', 'asset'],
     'Object2': 'updatesthe organizational process assets',
     'Object2_Lemmatization': ['updatesthe',
      'organizational',
      'process',
      'asset'],
     'Relation': []}],
   'RuleType': 'AsaResultOf'}],
 [{'Cause': [{'Object 1': ' a project charter',
     'Object1_Lemmatization': [' ', 'a', 'proje